## GROUP 1 - Julia
* DEL CARPIO CUENCA, GABRIEL SEBASTIAN
* ESPINOSA CALDERON, MAURICIO GUSTAVO
* JAIME MARTINEZ, KEVIN OSWALDO
* MELLIZO ANTAZU, MILAGROS ESTEFANY
* QUISPE ROBLADILLO, ALMENDRA VALERIA

## **Assignment 2**


In this assignment, you will be creating your own cross-fitting procedure and apply it to the wages data that we have been using.

In [24]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Unrecognized magic `%%shell`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


In [25]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Statistics")
Pkg.add("StatsBase")
Pkg.add("StatsModels")
Pkg.add("GLM")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [26]:
using DataFrames
using Statistics
using StatsBase
using CSV
using StatsModels
using GLM

**1. Loading and processing the data**

1. Load the data, either from the url or from the path provided

In [27]:
data = CSV.read("/content/wage2015_subsample_inference (1).csv", DataFrame)
select!(data, Not("rownames"));

In [28]:
size(data)

(5150, 20)

In [29]:
describe(data, :nmissing)  # no hay missings

Row,variable,nmissing
,Symbol,Int64
1,wage,0
2,lwage,0
3,sex,0
4,shs,0
5,hsg,0
6,scl,0
7,clg,0
8,ad,0
9,mw,0


As in Group Assignment 1 2024 - 2 #1044 , generate the extra-flexible model. This means that it contains all two-way interactions between the experience polynomials and the indicator variables

In [30]:
describe(data, :min, :max, :mean, :std, :q25, :median, :q75)

Row,variable,min,max,mean,std,q25,median,q75
,Symbol,Real,Real,Float64,Float64,Float64,Float64,Float64
1,wage,3.02198,528.846,23.4104,21.003,13.4615,19.2308,27.7778
2,lwage,1.10591,6.2707,2.97079,0.570385,2.59984,2.95651,3.32424
3,sex,0.0,1.0,0.444466,0.496955,0.0,0.0,1.0
4,shs,0.0,1.0,0.023301,0.150872,0.0,0.0,0.0
5,hsg,0.0,1.0,0.243883,0.429465,0.0,0.0,0.0
6,scl,0.0,1.0,0.278058,0.448086,0.0,0.0,1.0
7,clg,0.0,1.0,0.31767,0.465616,0.0,0.0,1.0
8,ad,0.0,1.0,0.137087,0.343973,0.0,0.0,0.0
9,mw,0.0,1.0,0.259612,0.438464,0.0,0.0,1.0




2.1. Generate the array for the outcome variable Y and normalize it

2.2. Generate the array for the predictors X (do not generate an intercept) and normalize its colums.


$
\text{sex} + \text{(exp1} + \text{exp2} + \text{exp3} + \text{exp4} +
\text{hsg} + \text{scl} + \text{clg} + \text{ad} + \text{se} +
\text{we} + \text{ne} + \text{dummy(occ2)} + \text{dummy(ind2))} ^2
$

In [31]:

extra_flexible_formula = @formula(lwage ~ 1 + sex + (exp1 + exp2 + exp3 + exp4) * (hsg + scl + clg + ad + so + we + ne + occ2 + ind2) + (hsg + scl + clg + ad) * (so + we + ne + occ2 + ind2) + (so + we + ne) * (occ2 + ind2) + occ2 * ind2)
X_extra_flexible = modelmatrix(extra_flexible_formula, data);


In [32]:
size(X_extra_flexible)

(5150, 78)

3. Split between training and testing samples. The testing sample should be 10% of the total.

In [50]:
# Definir Y (variable dependiente) y X (matriz de predictores)
Y = data[!, :lwage]  # Por ejemplo, si 'lwage' es el log de los salarios
X = select(data, Not(:lwage))  # Seleccionar todas las columnas excepto 'lwage'

Row,wage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64
1,9.61538,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,3600.0,11,8370.0,18
2,48.0769,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,3050.0,10,5070.0,9
3,11.0577,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,6260.0,19,770.0,4
4,13.9423,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,420.0,1,6990.0,12
5,28.8462,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22
6,11.7308,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.01,0.001,0.0001,1650.0,5,7460.0,14
7,19.2308,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,17.64,74.088,311.17,5120.0,17,7280.0,14
8,19.2308,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,37.0,13.69,50.653,187.416,5240.0,17,5680.0,9
9,12.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,4040.0,13,8590.0,19


In [53]:
# Convertir X (DataFrame) a una matriz numérica
X_matrix = Matrix(X)

# Normalizar las columnas de X_matrix
X_normalized = (X_matrix .- mean(X_matrix, dims=1)) ./ std(X_matrix, dims=1)

# Normalizar Y (asumiendo que Y ya es un array numérico)
Y_normalized = (Y .- mean(Y)) ./ std(Y)


5150-element Vector{Float64}:
 -1.2402545501637507
  1.581415943901389
 -0.9952235701262617
 -0.5888284120346895
  0.6858352078834439
 -0.8916287370734166
 -0.02502721376252582
 -0.02502721376252582
 -0.8518460993196265
 -0.02502721376252582
 -0.20974552813450043
 -0.8490387289010607
 -0.8490387289010607
  ⋮
  0.018263935171735916
  0.14207082714988994
  1.4866251590094772
  0.25770849799681306
 -0.2591347787012701
  0.3661886075001642
 -0.6009613493971904
 -0.48781225590462607
  0.29461938662075393
  1.1902001226386991
  0.9199427728221882
 -0.20974552813450043

In [55]:
# Normalizar las columnas de X
X_normalized = (X_matrix .- mean(X_matrix)) ./ std(X_matrix)

# Normalizar Y (asumiendo que Y ya es un array numérico)
Y_normalized = (Y .- mean(Y)) ./ std(Y)



5150-element Vector{Float64}:
 -1.2402545501637507
  1.581415943901389
 -0.9952235701262617
 -0.5888284120346895
  0.6858352078834439
 -0.8916287370734166
 -0.02502721376252582
 -0.02502721376252582
 -0.8518460993196265
 -0.02502721376252582
 -0.20974552813450043
 -0.8490387289010607
 -0.8490387289010607
  ⋮
  0.018263935171735916
  0.14207082714988994
  1.4866251590094772
  0.25770849799681306
 -0.2591347787012701
  0.3661886075001642
 -0.6009613493971904
 -0.48781225590462607
  0.29461938662075393
  1.1902001226386991
  0.9199427728221882
 -0.20974552813450043

In [56]:
using Random

# Definir n como el número de filas en X_normalized
n = size(X_normalized, 1)

# Determinar cuántas observaciones se usarán para el conjunto de entrenamiento (90% del total)
n_train = Int(round(0.9 * n))

# Generar los índices aleatorios para el conjunto de entrenamiento
train_indices = shuffle(1:n)[1:n_train]

# Los índices restantes serán para el conjunto de prueba
test_indices = setdiff(1:n, train_indices)

# Dividir los datos en entrenamiento y prueba
X_train = X_normalized[train_indices, :]
X_test = X_normalized[test_indices, :]
Y_train = Y_normalized[train_indices]
Y_test = Y_normalized[test_indices]

# Verificar las dimensiones de los conjuntos de entrenamiento y prueba
size(X_train), size(X_test)



((4635, 19), (515, 19))

## 2. The Lasso Cross-Validation Procedure

4. Program a function that generates a logarithmically spaced grid. The input arguments should be the lower and upper bounds of the grid, as well as the natural logarithm of the spacing between each element of the grid. The output should be the logarithmically spaced grid, meaning that if we take the natural logarithm of each entry in the grid, they will be equally spaced. This will be the grid of values for
λ
 values to try during cross-validation.

In [57]:

function log_grid(lower_limit::Float64, upper_limit::Float64, log_spacing::Float64)

    n_points = Int(round((log(upper_limit) - log(lower_limit)) / log_spacing)) + 1
    grid = exp.(range(log(lower_limit), stop=log(upper_limit), length=n_points))
    return grid
end


lower_limit = 1e-3
upper_limit = 1e3
log_spacing = 0.5

grid = log_grid(lower_limit, upper_limit, log_spacing)
println(grid)


[0.0010000000000000002, 0.0016378937069540646, 0.0026826957952797263, 0.004393970560760792, 0.0071968567300115215, 0.011787686347935875, 0.019306977288832503, 0.0316227766016838, 0.051794746792312114, 0.08483428982440722, 0.13894954943731377, 0.22758459260747885, 0.372759372031494, 0.6105402296585328, 1.0, 1.6378937069540642, 2.6826957952797255, 4.393970560760791, 7.19685673001152, 11.787686347935873, 19.3069772888325, 31.62277660168379, 51.794746792312104, 84.8342898244072, 138.94954943731375, 227.58459260747878, 372.75937203149397, 610.5402296585327, 999.9999999999998]


5. Program a function to generate
k
 folds. It should take as input the array to be split rowwise and the number of folds desired. It should output a list of
k
 1d arrays of booleans; these arrays should all be the same length as the number of rows in the input array, and when they are all summed together they should add up to an array of all true values. Create your own procedure for splitting. You can aid yourself with third party packages like numpy in Python or Statistics in Julia, but do not use a pre-programmed third party splitting procedure like sk-learns's KFolds in Python.

In [58]:

function generate_k_folds(n_rows::Int, k::Int)

    indices = shuffle(1:n_rows)


    fold_size = floor(Int, n_rows / k)
    folds = []

    for i in 1:k

        if i == k

            push!(folds, indices[(i-1)*fold_size+1:end])
        else
            push!(folds, indices[(i-1)*fold_size+1:i*fold_size])
        end
    end

    return folds
end

# Ejemplo de uso
n_rows = size(X_train, 1)
k = 5
folds = generate_k_folds(n_rows, k)


for i in 1:k
    println("Fold $i tiene tamaño: ", length(folds[i]))
end



Fold 1 tiene tamaño: 927
Fold 2 tiene tamaño: 927
Fold 3 tiene tamaño: 927
Fold 4 tiene tamaño: 927
Fold 5 tiene tamaño: 927


6. Program a function that integrates those that you programmed in the last two items to find the value of λ that minimizes the testing mean square error across folds. It should take the following inputs:

* Y: an array for the outcome variable.
* X: an array of predictors.
* lambda_bounds: the lower and upper bounds of the grid of lambda values.
* k: number of folds

The output should be a dictionary (a list in R) with the following entries:

* optimal_lambda: The lambda that minimizes the testing MSE across folds.
* optimal_coef: An array with the coefficients found for the optimal lambda
* all_lambdas: The grid of lambdas.
* all_mse: An array with the testing MSE across folds for each lambda.

The procedure goes as follows:

* With each lambda in the grid
*   For each split
*      Train and test a lasso model.



With this, you can get the testing MSE associated with the value of lambda in each iteration. Feel free to use a third party Lasso estimator, as long as it is not one that calculates the optimal lambda value. Instead, the third party estimator you use should only estimate the regression with the penalty weight provided so you can get the testing MSE with that specific weight.

In [59]:
import Pkg
Pkg.add("HDMjl")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [60]:
using HDMjl


function cross_validate_lasso(X, Y, lambdas, k_folds)
    n_lambdas = length(lambdas)
    mse_per_lambda = zeros(n_lambdas)

    for (i, lambda) in enumerate(lambdas)
        mse_folds = []


        for fold in k_folds

            train_indices = setdiff(1:size(X, 1), fold)
            test_indices = fold

            X_train, X_test = X[train_indices, :], X[test_indices, :]
            Y_train, Y_test = Y[train_indices], Y[test_indices]


            lasso_model = fit_lasso(X_train, Y_train, lambda)


            Y_pred = X_test * coef(lasso_model)[2:end]


            push!(mse_folds, mse(Y_test, Y_pred))
        end


        mse_per_lambda[i] = mean(mse_folds)
    end

    optimal_lambda_index = argmin(mse_per_lambda)
    optimal_lambda = lambdas[optimal_lambda_index]

    return (optimal_lambda, mse_per_lambda, optimal_lambda_index)
end





cross_validate_lasso (generic function with 1 method)

In [61]:
import Pkg
Pkg.add("GLMNet")



   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [62]:

function cross_validate_lasso(X, Y, lambdas, k_folds)
    n_lambdas = length(lambdas)
    mse_per_lambda = zeros(n_lambdas)


    for (i, lambda) in enumerate(lambdas)
        mse_folds = []


        for fold in k_folds

            train_indices = setdiff(1:size(X, 1), fold)
            test_indices = fold

            X_train, X_test = X[train_indices, :], X[test_indices, :]
            Y_train, Y_test = Y[train_indices], Y[test_indices]


            lasso_model = GLMNet.glmnet(X_train, Y_train, lambda=lambda, alpha=1.0)


            Y_pred = GLMNet.predict(lasso_model, X_test)


            push!(mse_folds, mse(Y_test, Y_pred))
        end


        mse_per_lambda[i] = mean(mse_folds)
    end


    optimal_lambda_index = argmin(mse_per_lambda)
    optimal_lambda = lambdas[optimal_lambda_index]

    return (optimal_lambda, mse_per_lambda, optimal_lambda_index)
end



cross_validate_lasso (generic function with 1 method)

7. Program a function for predicting the outcome variable through model estimated with the optimal lambda. It should take as inputs

* optimal_model: A dictionary with the values outputed by the function defined for the previous point.
* X: an array of predictors.

The output should be an array of predicted values.

In [63]:

function cross_validate_lasso(X, Y, lambdas, k_folds)
    n_lambdas = length(lambdas)
    mse_per_lambda = zeros(n_lambdas)


    for (i, lambda) in enumerate(lambdas)
        mse_folds = []


        for fold in k_folds

            train_indices = setdiff(1:size(X, 1), fold)
            test_indices = fold

            X_train, X_test = X[train_indices, :], X[test_indices, :]
            Y_train, Y_test = Y[train_indices], Y[test_indices]

            lasso_model = GLMNet.glmnet(X_train, Y_train, lambda=[lambda], alpha=1.0)


            Y_pred = GLMNet.predict(lasso_model, X_test)


            push!(mse_folds, mse(Y_test, Y_pred))
        end


        mse_per_lambda[i] = mean(mse_folds)
    end


    optimal_lambda_index = argmin(mse_per_lambda)
    optimal_lambda = lambdas[optimal_lambda_index]

    return (optimal_lambda, mse_per_lambda, optimal_lambda_index)
end


cross_validate_lasso (generic function with 1 method)

In [66]:
using GLMNet  # Importar el paquete para ajuste Lasso y Ridge


In [69]:
# Definir la función para calcular el Mean Squared Error (MSE)
function mse(y_true, y_pred)
    return mean((y_true - y_pred).^2)
end


mse (generic function with 1 method)

In [70]:
# Definir la función MSE
function mse(y_true, y_pred)
    return mean((y_true - y_pred).^2)
end

# Generar la grilla de valores de lambda (lambdas)
lambdas = log_grid(1e-7, 1e7, 0.1)  # Ajusta los límites si es necesario

# Ejecutar la validación cruzada para encontrar el mejor lambda
optimal_lambda, mse_per_lambda, optimal_lambda_index = cross_validate_lasso(X_train, Y_train, lambdas, folds)

# Mostrar el mejor valor de lambda
println("El mejor valor de λ es: ", optimal_lambda)

# Ajustar el modelo Lasso con el valor óptimo de lambda
lasso_model_final = GLMNet.glmnet(X_train, Y_train, lambda=[optimal_lambda], alpha=1.0)

# Hacer predicciones en el conjunto de prueba
Y_pred_test = GLMNet.predict(lasso_model_final, X_test)

# Calcular el MSE en el conjunto de prueba
mse_test = mse(Y_test, Y_pred_test)

# Mostrar el MSE
println("MSE en el conjunto de prueba: ", mse_test)


El mejor valor de λ es: 0.0002721338768375309
MSE en el conjunto de prueba: 0.3282158714000657


## 3. Applying the Lasso Cross-Validation Procedure

8. Fit a simple OLS model with the training sample.

In [76]:
# Definir la función para calcular el R² (Coeficiente de determinación)
function r_squared(y_true, y_pred)
    ss_total = sum((y_true .- mean(y_true)).^2)
    ss_residual = sum((y_true .- y_pred).^2)
    return 1 - (ss_residual / ss_total)
end



r_squared (generic function with 1 method)

In [77]:
using DataFrames, GLM

# Definir la función para calcular el R²
function r_squared(y_true, y_pred)
    ss_total = sum((y_true .- mean(y_true)).^2)
    ss_residual = sum((y_true .- y_pred).^2)
    return 1 - (ss_residual / ss_total)
end

# Convertir los datos en DataFrame y generar nombres de columnas automáticamente
train_df = DataFrame(X_train, :auto)
train_df.Y_train = Y_train  # Añadir la variable dependiente al DataFrame

# Crear la fórmula automática para OLS (usando todos los predictores)
predictor_names = names(train_df[!, Not(:Y_train)])  # Obtener los nombres de las columnas predictor
formula_string = "Y_train ~ " * join(predictor_names, " + ")
ols_formula = @eval @formula($(Meta.parse(formula_string)))

# Ajustar el modelo OLS con la fórmula generada
ols_model = lm(ols_formula, train_df)

# Convertir X_test en DataFrame con nombres de columnas automáticamente
test_df = DataFrame(X_test, :auto)

# Hacer predicciones en el conjunto de prueba con el modelo OLS usando GLM.predict
Y_pred_ols = GLM.predict(ols_model, test_df)

# Calcular el MSE en el conjunto de prueba para el modelo OLS
mse_ols_test = mse(Y_test, Y_pred_ols)

# Calcular el R² para el modelo OLS
r2_ols = r_squared(Y_test, Y_pred_ols)

# Mostrar el MSE y R² del modelo OLS
println("MSE del modelo OLS: ", mse_ols_test)
println("R² del modelo OLS: ", r2_ols)


MSE del modelo OLS: 0.32868861330429106
R² del modelo OLS: 0.6998361615191326


9. Find the optimal lambda and its related coefficients with the function programmed in 3. and the training sample. Print the lambda and the coefficients.

10. Now use each language's hdm package to fit a Lasso model with the theoretical optimal lambda value. You can find the documentation of the package for each language in the following links


Ahora utilice el paquete hdm de cada lenguaje para ajustar un modelo Lasso con el valor lambda óptimo teórico. Puede encontrar la documentación del paquete para cada lenguaje en los siguientes enlaces

R: https://cran.r-project.org/web/packages/hdm/index.html
Python: https://github.com/maxhuppertz/hdmpy
Julia: https://github.com/d2cml-ai/HDMjl.jl

11. Report the testing MSE and R^2 for the OLS model and, the cross-validation lambda Lasso model, and the hdm theoretical lambda model. Use the function programmed in 4. to compute the predictions of the cross-validation Lasso model.